[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pdf-tools/components-code-sample-hub/blob/main/jupyter/pdftools_toolbox/pdftools_toolbox_merge_with_outlines.ipynb)

In [ ]:
%pip install https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/productkits/PDFSDKXT/latest/pdftools_toolbox-latest.tar.gz
%pip install ipython

# Merge multiple PDFs with outlines
Merge several PDF documents to one, while creating an
outline item for each input document.

In [ ]:
import io
import os
from pdftools_toolbox.pdf import Document, PageCopyOptions, PageList
from pdftools_toolbox.pdf.navigation import LocationZoomDestination, OutlineCopyOptions, OutlineItem

In [ ]:
# Download a file from a given URL and save it to the local system
def prepare_file(url: str, path: str):
    import requests
    response = requests.get(url)
    response.raise_for_status()

    with open(path, 'wb') as f:
        f.write(response.content)

In [ ]:
# Set input arguments
input_url1 = 'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/GraphicsNone.pdf'
input_url2 = 'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/BlankNoneNoTP.pdf'
input_paths = ['GraphicsNone.pdf', 'BlankNoneNoTP.pdf']
prepare_file(input_url1, input_paths[0])
prepare_file(input_url2, input_paths[1])
output_file_path = 'MergedWithOutlines.pdf'

In [ ]:
try:
    # Set and check license key. If the license key is not valid, an exception is thrown.
    from pdftools_toolbox.sdk import Sdk
    Sdk.initialize("<PDFSDK,V1,MGAASQD6L2JMQHL54PK08RQX8GG4SS0M8DAHVPH0VMP3NB8R9DUK>", None)

    # Create output document
    with open(output_file_path, "wb+") as out_stream:
        with Document.create(out_stream, None, None) as out_doc:
    
            # Define page copy options
            page_copy_options = PageCopyOptions()
            page_copy_options.copy_outline_items = False
    
            # Define outline copy options
            outline_copy_options = OutlineCopyOptions()
    
            # Get output pages
            out_pages = out_doc.pages
    
            # Merge input documents
            for input_path in input_paths:
                # Open input document
                with open(input_path, "rb") as in_stream:
                    with Document.open(in_stream, None) as in_doc:
    
                        # Copy all pages and append to output document
                        copied_pages = PageList.copy(out_doc, in_doc.pages, page_copy_options)
                        out_pages.extend(copied_pages)
    
                        # Create outline item
                        title = in_doc.metadata.title or os.path.basename(input_path)
                        first_copied_page = copied_pages[0]
                        destination = LocationZoomDestination.create(out_doc, first_copied_page, 0, first_copied_page.size.height, None)
                        outline_item = OutlineItem.create(out_doc, title, destination)
                        out_doc.outline.append(outline_item)
    
                        # Add outline items from input document as children
                        children = outline_item.children
                        for in_outline_item in in_doc.outline:
                            children.append(OutlineItem.copy(out_doc, in_outline_item, outline_copy_options))

    print("Execution successful.")
except Exception as e:
    print(f"An error occurred: {e}")